# Santa's Stolen Sleigh : A Heuristic Approach

Kaggle challenge link: https://www.kaggle.com/c/santas-stolen-sleigh

This solution relies on the key insight that mo matter how, all gifts must be carried down from the North Pole to their destination. At the very least, all gifts must travel along geodesics. This sets a hard minimum on the total work required to deliver all the gifts. Accounting for the return trips sets a higher, soft minimum, since it is hard to predict what the return trips may look like in an ideal solution.

Another insight is that it is beneficial to limit the number of trips returninig from Antarctica. The set of gifts is then divided into Antarctica and the rest of the world using the DBSCAN clustering algorithm and some manual adjustments.

The paths can be kept as straight as possible by forcing lateral displacements to be as small as possible for each path. This leads to the longitudinal binning algorithm. Both sets of gifts (Antartica + rest of the world) are ordered by longitude, and then binned from the start of hte gift list, closing a bin everytime the total trip weight would spill over the allowed 990 pounds.

Finally, optimizers are deisgned to fine-tune the solution. One algorithm tries to find a more optimal order of deliver for each trip. A second algorithm tries to find exchanges between neighboring trips that brings down the combined work of both trips. A third algorithm steals gifts for neighboring trips up to the weight limit, trying to bring down the combined work of the trip and its neighbors.

## Initial preparation: import statements and configuration

In [1]:
## Enable matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors

## Imports
import pandas as pd
pd.set_option('mode.chained_assignment',None)
pd.set_option('display.mpl_style', 'default') 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import numpy as np
from sklearn.cluster import DBSCAN

import math, copy, time, random

from cv2 import imread, cvtColor, COLOR_BGR2RGB

## Convenience functions

### Haversine
The haversine is the metric that measures distance between two points on a sphere. This function returns the distance on Earth, assuming a uniform radius of 6371 km.

In [2]:
## --------------------------------------------------
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    
    sklearn implementation
    2 arcsin(sqrt(sin^2(0.5*dx)cos(x1)cos(x2)sin^2(0.5*dy)))
    """
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    return c * r

### Trip work
A function that calculates the work for a single trip.

In [3]:
## --------------------------------------------------
def trip_work(trip):
    """
    Calculates the work for a trip
    """
    
    work = 0.0
    total_weight = 10.0 + trip['Weight'].sum()
    lon = 0
    lat = 90
    
    for i, row in trip.iterrows():
        current_lon = row['Longitude']
        current_lat = row['Latitude']
        current_w   = row['Weight']
        
        distance = haversine(lon, lat, current_lon, current_lat)
        work += distance * total_weight
        
        total_weight -= current_w
        lon = current_lon
        lat = current_lat
        
    work += haversine(lon, lat, 0, 90) * 10.0
    
    return work

### Total work
A function that calculates the work for the entire set of gifts, which also tells you how many trips are in the solution.

In [4]:
## --------------------------------------------------
def total_work(df):
    """
    Calculates the total work on all gifts
    """
    
    n = df['TripId'].max()
    x = 0
    for i in range(1, n+1):
        trip = df[df['TripId'] == i]
        x += trip_work(trip)
    return x, n

### Longitudinal binning
Make trips by constructing bins in longitude in which the sum of the weights is as close as possible to 990 lbs. The trips are then ordered by reverse order of latitude, which is a suboptimal but pretty decent first attempt at finding a good order of delivery.

In [5]:
## --------------------------------------------------
def longitudinal_binning(df, maxd=360.0):
    """
    Make the trips by longitudinal binning and latitude ordering
    """
    
    df = df.sort_values(by='Longitude')
    
    ## Bin the longitude axis such that every bin sums up to 990 pounds
    trip_numbers = []
    trip_number = 1
    weight = 10.0
    longitude = gifts_df['Longitude'].values[0]
    max_delta_longitude = maxd

    for i, row in df.iterrows():
        current_weight = row['Weight']
        current_longitude = row['Longitude']
        if (weight + current_weight > 1000.0) or (current_longitude - longitude > max_delta_longitude):
            trip_number += 1
            weight = 10.0 + current_weight
            longitude = current_longitude
        else:
            weight += current_weight
        trip_numbers.append((row['GiftId'], trip_number))
    
    trip_numbers = np.array(trip_numbers, dtype=int)
    
    trip_numbers = pd.DataFrame(trip_numbers, columns=['GiftId', 'TripId'])
    
    df = pd.merge(df, trip_numbers)
    
    df = df.sort_values(by=['TripId','Latitude'], ascending=[True,False])
    
    return df

## Optimization functions

### Trip optimizer
The trip optimizer starts with the heaviest, further down gift to deliver of the entire trip. It then iterates through the rest of the gifts, in descending order of work, and finds the best position to insert them in the trip in order to increase the score minimally. 

In [6]:
## --------------------------------------------------
def optimize_trips(gifts_df):
    """
    Optimize the order of delivery in each trip, treating every trip as closed
    """

    n = len(gifts_df['TripId'].unique())
    cumulative_improvement = 0

    for i in xrange(1,n+1):
    
        start_time = time.time()
    
        ## Obtain trip and calculate the initial work
        trip = gifts_df[gifts_df['TripId'] == i]
        initial_trip_work = trip_work(trip)
    
        ## Give optimization priority to heavier weights that are further down
        trip.sort_values(by=['Weight', 'Latitude'], ascending=[False,True], inplace=True)
    
        ## Start the revised trip with the first entry, make a list of the gifts to append
        revised_trip = trip[:1]
        rest_trip = trip[1:]
    
        ## Make an order row
        n_revised = 1
        revised_trip['order'] = np.arange(1,n_revised+1)
    
        for index, row in rest_trip.iterrows():
            ## Generate intermediary indices for the difference insertion positions
            inserts = [j+0.5 for j in xrange(n_revised+1)]
            row_df = row.to_frame().transpose()
    
            best_work = float('inf')
            best_trip = None
    
            for k in inserts:
                row_df['order'] = k
                test_trip = pd.concat([revised_trip, row_df])
                test_trip = test_trip.sort_values(by='order')
                work = trip_work(test_trip)
                if work < best_work:
                    best_work = work
                    best_trip = test_trip
            
            revised_trip = best_trip
            n_revised = len(revised_trip)
            revised_trip['order'] = np.arange(1,n_revised+1)    
    
        ## Calculate the final trip work and the cumulative improvement
        final_trip_work = trip_work(revised_trip)
    
        if final_trip_work < initial_trip_work:
            cumulative_improvement += initial_trip_work - final_trip_work
        
            ## Put the trip back into the gifts DF
            del revised_trip['order']
            gifts_df[gifts_df['TripId'] == i] = revised_trip.values
        
        else:
            print 'Failed at improving:', initial_trip_work - final_trip_work
        
        end_time = time.time()
        print 'Trip', i, 'cumulative improvement so far:', cumulative_improvement, 'dt:', end_time - start_time
        
    return gifts_df

### Pair exchange optimizer
This optimizer looks at every possible pairing of gifts from two neighboring trips and swaps them. As long as the swap is permitted by the weight budget of each trip, and as long as the swap decreases the combined work of the two trips, the swap is kept.

In [7]:
## --------------------------------------------------
def optimize_pair(tripA, tripB):
    """
    Try all exchanges between trips, keep the exchanges that reduces the total work
    """
    
    total_work = trip_work(tripA) + trip_work(tripB)
    
    nA = len(tripA)
    nB = len(tripB)

    for jA, rowA in tripA.iterrows():
        for jB, rowB in tripB.iterrows():
                
            rA = copy.copy(tripA.loc[jA].values)
            rB = copy.copy(tripB.loc[jB].values)

            tripA.loc[jA] = rB
            tripB.loc[jB] = rA
            
            new_work = trip_work(tripA) + trip_work(tripB)
            wA = tripA['Weight'].sum()
            wB = tripB['Weight'].sum()
        
            ## If the new work is better, keep the exchange and change the total work to beat
            if new_work < total_work and (wA < 990) and (wB < 990):
                total_work = new_work
            ## If the new work isn't better, undo the exchange
            else:
                tripA.loc[jA] = rA
                tripB.loc[jB] = rB
                

## --------------------------------------------------
def optimize_pairs(gifts_df):
    """
    Optimize pairs of trips by swapping gifts between them
    """
    
    improvement = 0
    n = gifts_df['TripId'].max()

    t = time.time()

    for i in range(n-1):
        iA = i+1
        if i==0: iA == n
        iB = i+2
    
        A = gifts_df[gifts_df['TripId'] == iA]
        B = gifts_df[gifts_df['TripId'] == iB]
    
        before = trip_work(A) + trip_work(B)
    
        optimize_pair(A,B)
    
        after = trip_work(A) + trip_work(B)
    
        improvement += before-after
    
        A['TripId'] = iA
        B['TripId'] = iB
    
        gifts_df[gifts_df['TripId'] == iA] = A
        gifts_df[gifts_df['TripId'] == iB] = B
    
        current_time = time.time()
        print i+1, 'Cumulative improvement:', improvement, 'dt:', current_time - t
        t = time.time()
        
    return gifts_df

### Steal optimizer
Optimize trips by stealing gifts from neighboring trips until the mass budget is met.

In [8]:
## --------------------------------------------------
def optimize_steal(gifts_df):
    """
    Optimize trips by stealing from neighbors
    """
    
    cumulative_improvement = 0
    n = gifts_df['TripId'].max()

    for i in range(1,n-1):
        trip_before = gifts_df[gifts_df['TripId'] == i]
        trip        = gifts_df[gifts_df['TripId'] == i+1]
        trip_after  = gifts_df[gifts_df['TripId'] == i+2]
    
        n_trip = len(trip)
    
        trip['order'] = np.arange(1,n_trip+1)
        inserts = [j+0.5 for j in xrange(n_trip+1)]
    
        mass_budget = 990 - trip['Weight'].sum()
        print 'Trip', i, 'mass budget:', mass_budget
        if mass_budget < 0: continue
    
        work_trip_before = trip_work(trip_before)
        work_trip        = trip_work(trip)
        work_trip_after  = trip_work(trip_after)
    
        initial_work = work_trip_before + work_trip + work_trip_after
    
        best_work = work_trip_before + work_trip
    
        for index, row in trip_before.iterrows():
        
            gift = int(row['GiftId'])
            mass = row['Weight']
            if mass_budget - mass < 0: continue
            row_df = row.to_frame().transpose()
            test_trip_before = trip_before[trip_before['GiftId'] != gift]
            work_trip_before = trip_work(test_trip_before)
        
            for k in inserts:
                row_df['order'] = k
                test_trip = pd.concat([trip, row_df])
                test_trip = test_trip.sort_values(by='order')
                work = trip_work(test_trip) + work_trip_before
                if work < best_work:
                    best_work = work
                    trip = test_trip
                    trip_before = test_trip_before
                    mass_budget -= mass
                    break
                
        best_work = trip_work(trip_after) + trip_work(trip)
                
        for index, row in trip_after.iterrows():
        
            gift = int(row['GiftId'])
            mass = row['Weight']
            if mass_budget - mass < 0: continue
            row_df = row.to_frame().transpose()
            test_trip_after = trip_after[trip_after['GiftId'] != gift]
            work_trip_after = trip_work(test_trip_after)
        
            for k in inserts:
                row_df['order'] = k
                test_trip = pd.concat([trip, row_df])
                test_trip = test_trip.sort_values(by='order')
                work = trip_work(test_trip) + work_trip_after
                if work < best_work:
                    best_work = work
                    trip = test_trip
                    trip_after = test_trip_after
                    mass_budget -= mass
                    break
                
        work_trip_before = trip_work(trip_before)
        work_trip        = trip_work(trip)
        work_trip_after  = trip_work(trip_after)
    
        final_work = work_trip_before + work_trip + work_trip_after
    
        improvement = initial_work - final_work
    
        if improvement > 0:
        
            del trip['order']
            trip_before['TripId'] = i
            trip['TripId'] = i+1
            trip_after['TripId'] = i+2
        
            values = pd.concat([trip_before, trip, trip_after])
        
            mask = (gifts_df['TripId'] > (i-1)) & (gifts_df['TripId'] < (i+3))
            gifts_df[mask] = values.values
        
            cumulative_improvement += improvement
        
        print i, 'Cumulative improvement:', cumulative_improvement
        
    return gifts_df

### Straight down trip optimizer

In [9]:
## --------------------------------------------------
def optimize_straight_down(gifts_df):
    """
    Try to find out if re-ordering the gifts by latitude improves anything
    """
    
    n = len(gifts_df['TripId'].unique())
    cumulative_improvement = 0

    for i in xrange(1,n+1):
        trip = gifts_df[gifts_df['TripId'] == i]
        
        current_work = trip_work(trip)
        straight_down_trip = trip.sort_values(by='Latitude', ascending=False)
        straight_down_work = trip_work(straight_down_trip)
        
        if straight_down_work < current_work:
            cumulative_improvement += current_work - straight_down_work
            gifts_df[gifts_df['TripId'] == i] = straight_down_trip.values
            
        print i, 'Cumulative improvement:', cumulative_improvement
            
    return gifts_df
    

## Execution

### Clustering and longitudinal binning

In [10]:
## Load data
gifts_df = pd.read_csv('gifts.csv')

## Convert longitude and latitude to radians in new columns
gifts_df['lon_rad'] = np.deg2rad(gifts_df['Longitude'].values)
gifts_df['lat_rad'] = np.deg2rad(gifts_df['Latitude'].values)

## Calculate minimal cost for each gift
gifts_df['NPlon'] = 0
gifts_df['NPlat'] = 90
gifts_df['Cost'] = gifts_df['Weight'] * map(haversine, gifts_df['NPlon'], gifts_df['NPlat'], gifts_df['Longitude'], gifts_df['Latitude'])

In [11]:
## Frame the DBSCAN parameters in terms of the problem
earth_radius = 6371.0   # km
minimum_distance = 750.0 # km
eps = minimum_distance/earth_radius

## Do the clustering
clustering = DBSCAN(eps=eps, min_samples=5, metric='haversine')
gifts_df['cluster'] = clustering.fit_predict(gifts_df[['lat_rad', 'lon_rad']].values)

## Making Antarctica into one cluster, everything else together
m = {-1:0, 0:0, 1:1, 2:0, 3:0, 4:0, 5:0}
gifts_df['cluster'] = gifts_df['cluster'].map(m)

In [12]:
clusters = []
n_gifts_covered = 0

## Partition the dataframe in clusters
for i in range(2):
    cluster = gifts_df[gifts_df['cluster'] == i]
    cluster = longitudinal_binning(cluster)
    cluster['TripId'] = cluster['TripId'] + n_gifts_covered
    n_gifts_covered = cluster['TripId'].max()
    clusters.append(cluster)
    
gifts_df = pd.concat(clusters)

In [13]:
del gifts_df['lon_rad']
del gifts_df['lat_rad']
del gifts_df['NPlon']
del gifts_df['NPlat']

In [14]:
x,n = total_work(gifts_df)
print 'Total work:', x, 'in', n, 'trips'

Total work: 12527494345.7 in 1446 trips


### Optimization

In [15]:
gifts_df = optimize_trips(gifts_df)

Trip 1 cumulative improvement so far: 1439652.02601 dt: 10.4094181061
Trip 2 cumulative improvement so far: 3861326.87462 dt: 12.9535400867
Trip 3 cumulative improvement so far: 4384873.66105 dt: 8.46168208122
Trip 4 cumulative improvement so far: 4533400.32719 dt: 5.29388594627
Trip 5 cumulative improvement so far: 4730686.57331 dt: 5.52564501762
Trip 6 cumulative improvement so far: 4865845.37375 dt: 11.7262811661
Trip 7 cumulative improvement so far: 4877380.26431 dt: 6.06257796288
Trip 8 cumulative improvement so far: 4889892.2951 dt: 5.58038902283
Trip 9 cumulative improvement so far: 4969294.63456 dt: 12.1948428154
Trip 10 cumulative improvement so far: 5503430.01118 dt: 12.1066179276
Trip 11 cumulative improvement so far: 5664594.90368 dt: 7.8190908432
Trip 12 cumulative improvement so far: 5848853.77547 dt: 7.67021107674
Trip 13 cumulative improvement so far: 6477011.60329 dt: 11.0778398514
Trip 14 cumulative improvement so far: 7231912.39204 dt: 12.0906071663
Trip 15 cumulativ

In [16]:
gifts_df = optimize_steal(gifts_df)

Trip 1 mass budget: 1.56652570801
1 Cumulative improvement: 0
Trip 2 mass budget: 3.9750235136
2 Cumulative improvement: 326.741877723
Trip 3 mass budget: 24.1614036655
3 Cumulative improvement: 14134.3092097
Trip 4 mass budget: 10.55980537
4 Cumulative improvement: 29580.9090998
Trip 5 mass budget: 30.6829550074
5 Cumulative improvement: 122870.20794
Trip 6 mass budget: 45.5416641631
6 Cumulative improvement: 134854.467053
Trip 7 mass budget: 39.4663856711
7 Cumulative improvement: 157360.746623
Trip 8 mass budget: 27.3116873827
8 Cumulative improvement: 165450.862962
Trip 9 mass budget: 0.74027330012
9 Cumulative improvement: 165450.862962
Trip 10 mass budget: 8.50183614137
10 Cumulative improvement: 171572.92868
Trip 11 mass budget: 4.56857800783
11 Cumulative improvement: 174384.881907
Trip 12 mass budget: 20.249894187
12 Cumulative improvement: 184845.972885
Trip 13 mass budget: 21.0053981593
13 Cumulative improvement: 187507.689124
Trip 14 mass budget: 23.0178061675
14 Cumulative

In [17]:
gifts_df = optimize_pairs(gifts_df)

1 Cumulative improvement: 660.872604139 dt: 37.0043458939
2 Cumulative improvement: 660.872604139 dt: 28.0213010311
3 Cumulative improvement: 2563.51879448 dt: 19.7116429806
4 Cumulative improvement: 13787.2660153 dt: 18.4378049374
5 Cumulative improvement: 14245.6169317 dt: 26.5514090061
6 Cumulative improvement: 199349.119815 dt: 28.6290249825
7 Cumulative improvement: 199970.196676 dt: 18.4846699238
8 Cumulative improvement: 204143.430199 dt: 27.7726280689
9 Cumulative improvement: 204143.430199 dt: 38.1140058041
10 Cumulative improvement: 204781.724423 dt: 32.2456891537
11 Cumulative improvement: 204985.166606 dt: 27.8384628296
12 Cumulative improvement: 209447.657345 dt: 32.7619049549
13 Cumulative improvement: 209447.657345 dt: 38.2476189137
14 Cumulative improvement: 209447.657345 dt: 36.4013090134
15 Cumulative improvement: 212677.407717 dt: 24.7669699192
16 Cumulative improvement: 212677.407717 dt: 18.7690179348
17 Cumulative improvement: 212677.407717 dt: 27.6082499027
18 Cum

In [18]:
gifts_df = optimize_straight_down(gifts_df)

1 Cumulative improvement: 0
2 Cumulative improvement: 0
3 Cumulative improvement: 0
4 Cumulative improvement: 0
5 Cumulative improvement: 0
6 Cumulative improvement: 0
7 Cumulative improvement: 0
8 Cumulative improvement: 0
9 Cumulative improvement: 0
10 Cumulative improvement: 0
11 Cumulative improvement: 0
12 Cumulative improvement: 0
13 Cumulative improvement: 0
14 Cumulative improvement: 0
15 Cumulative improvement: 0
16 Cumulative improvement: 0
17 Cumulative improvement: 0
18 Cumulative improvement: 0
19 Cumulative improvement: 0
20 Cumulative improvement: 0
21 Cumulative improvement: 0
22 Cumulative improvement: 0
23 Cumulative improvement: 0
24 Cumulative improvement: 0
25 Cumulative improvement: 0
26 Cumulative improvement: 0
27 Cumulative improvement: 0
28 Cumulative improvement: 0
29 Cumulative improvement: 0
30 Cumulative improvement: 0
31 Cumulative improvement: 0
32 Cumulative improvement: 0
33 Cumulative improvement: 0
34 Cumulative improvement: 0
35 Cumulative improveme

## Evaluate and save

In [19]:
x,n = total_work(gifts_df)
print 'Total work:', x, 'in', n, 'trips'

Total work: 12475104040.3 in 1446 trips


In [20]:
trips_df = gifts_df[['GiftId', 'TripId']]
trips_df.to_csv('trips.csv', index=False)